In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

%matplotlib inline

In [3]:
image_gen = ImageDataGenerator(rotation_range=20,
                               shear_range=0.2,
                               rescale=1/255,
                               zoom_range=0.2,
                               fill_mode='nearest',
                               width_shift_range=0.1,
                               height_shift_range=0.1)

In [81]:
image = cv2.imread("./Monkey_Pox_Data/train/Monkey_Pox/M01_01_09.jpg")
image.shape

(800, 1200, 3)

In [52]:
train_image_gen = image_gen.flow_from_directory("./Monkey_Pox_Data/train", batch_size=16, class_mode='binary', target_size=(200, 200))

test_image_gen = image_gen.flow_from_directory("./Monkey_Pox_Data/test", batch_size=16, class_mode='binary', target_size=(200, 200))

Found 3193 images belonging to 2 classes.
Found 228 images belonging to 2 classes.


In [55]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(200, 200, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
model.fit(train_image_gen, epochs=30, steps_per_epoch=150,
          validation_data=train_image_gen, validation_steps=12)

Epoch 1/30
150/150 [==============================] - 145s 962ms/step - loss: 0.5796 - accuracy: 0.7038 - val_loss: 0.5771 - val_accuracy: 0.7188
Epoch 2/30
150/150 [==============================] - 143s 953ms/step - loss: 0.5637 - accuracy: 0.7254 - val_loss: 0.5784 - val_accuracy: 0.7083
Epoch 3/30
150/150 [==============================] - 129s 860ms/step - loss: 0.5520 - accuracy: 0.7254 - val_loss: 0.5354 - val_accuracy: 0.7448
Epoch 4/30
150/150 [==============================] - 128s 854ms/step - loss: 0.5551 - accuracy: 0.7288 - val_loss: 0.5060 - val_accuracy: 0.7552
Epoch 5/30
150/150 [==============================] - 130s 866ms/step - loss: 0.5504 - accuracy: 0.7221 - val_loss: 0.4793 - val_accuracy: 0.8021
Epoch 6/30
150/150 [==============================] - 131s 869ms/step - loss: 0.5373 - accuracy: 0.7321 - val_loss: 0.4803 - val_accuracy: 0.7656
Epoch 7/30
150/150 [==============================] - 130s 864ms/step - loss: 0.5421 - accuracy: 0.7254 - val_loss: 0.6006 -

In [82]:
monkey_pox = load_img("./Skin_Cancer_Test.jpg", target_size=(200, 200))

monkey_pox = img_to_array(monkey_pox)

monkey_pox = monkey_pox / monkey_pox.max()

monkey_pox = np.expand_dims(monkey_pox, axis=0)

monkey_pox.shape

(1, 200, 200, 3)

In [83]:
predictions = model.predict(monkey_pox)

predictions

1/1 [==============================] - 0s 273ms/step


array([[0.0526654]], dtype=float32)

In [84]:
result = "nothing"

if predictions[0] < 0.5:
    result = "Monkey Pox"
else:
    result = "Others"
    
print(result)

Monkey Pox


In [68]:
model.save("Monkey_Pox(30).h5")